# SwellSight Loss Functions and Metrics (Standalone)

This notebook defines the multi-task loss function and evaluation metrics for the SwellSight model.
**This is a standalone version that includes all necessary code and doesn't require external files.**

## Components
- Multi-task loss combining regression and classification
- Regression metrics (MAE, MSE, RMSE)
- Classification metrics (accuracy, macro F1)
- Sample weighting based on confidence
- Comprehensive visualization tools

In [ ]:
# Install required packages if not available
import subprocess
import sys

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
required_packages = ['torch', 'matplotlib', 'seaborn', 'numpy', 'scikit-learn']
for package in required_packages:
    install_if_missing(package)

print("✓ All required packages are available")

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from typing import Dict, List, Tuple, Optional

print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## Multi-Task Loss Function

In [ ]:
class MultiTaskLoss(nn.Module):
    """Multi-task loss combining regression and classification losses.
    
    Combines:
    - Height regression loss (SmoothL1Loss)
    - Wave type classification loss (CrossEntropyLoss)
    - Direction classification loss (CrossEntropyLoss)
    
    All losses are weighted by sample confidence and task importance.
    """
    
    def __init__(self, w_height=1.0, w_wave_type=1.0, w_direction=1.0):
        """
        Args:
            w_height: Weight for height regression loss
            w_wave_type: Weight for wave type classification loss
            w_direction: Weight for direction classification loss
        """
        super().__init__()
        self.w_height = w_height
        self.w_wave_type = w_wave_type
        self.w_direction = w_direction

        # Use SmoothL1Loss for height (robust to outliers)
        self.reg = nn.SmoothL1Loss(reduction="none")
        # Use CrossEntropyLoss for classifications
        self.ce = nn.CrossEntropyLoss(reduction="none")

    def forward(self, pred_h, pred_wt, pred_dir, y_h, y_wt, y_dir, sample_w):
        """
        Args:
            pred_h: Height predictions (batch_size, 1)
            pred_wt: Wave type logits (batch_size, num_wave_types)
            pred_dir: Direction logits (batch_size, num_directions)
            y_h: True heights (batch_size, 1)
            y_wt: True wave type labels (batch_size,)
            y_dir: True direction labels (batch_size,)
            sample_w: Sample weights (batch_size, 1)
            
        Returns:
            Tuple of (total_loss, loss_dict)
        """
        w = sample_w.view(-1)

        # Height regression loss
        loss_h = self.reg(pred_h.view(-1), y_h.view(-1))
        loss_h = (loss_h * w).mean()

        # Wave type classification loss
        loss_wt = self.ce(pred_wt, y_wt.view(-1))
        loss_wt = (loss_wt * w).mean()

        # Direction classification loss
        loss_dir = self.ce(pred_dir, y_dir.view(-1))
        loss_dir = (loss_dir * w).mean()

        # Combine losses with task weights
        total = self.w_height * loss_h + self.w_wave_type * loss_wt + self.w_direction * loss_dir
        
        loss_dict = {
            "loss_h": float(loss_h.item()), 
            "loss_wt": float(loss_wt.item()), 
            "loss_dir": float(loss_dir.item()),
            "total_loss": float(total.item())
        }
        
        return total, loss_dict

print("✓ MultiTaskLoss class defined")

## Regression Metrics

In [ ]:
def regression_metrics(y_true: torch.Tensor, y_pred: torch.Tensor) -> dict:
    """
    Compute common regression metrics for wave height prediction.
    
    Args:
        y_true: True values (N,) or (N, 1)
        y_pred: Predicted values (N,) or (N, 1)
        
    Returns:
        Dictionary with mae, mse, rmse, r2
    """
    y_true = y_true.detach().float().view(-1)
    y_pred = y_pred.detach().float().view(-1)

    # Basic error metrics
    err = y_pred - y_true
    mae = err.abs().mean()
    mse = (err ** 2).mean()
    rmse = torch.sqrt(mse)
    
    # R-squared (coefficient of determination)
    ss_res = ((y_true - y_pred) ** 2).sum()
    ss_tot = ((y_true - y_true.mean()) ** 2).sum()
    r2 = 1 - (ss_res / (ss_tot + 1e-8))

    return {
        "mae": mae.item(),
        "mse": mse.item(),
        "rmse": rmse.item(),
        "r2": r2.item(),
        "mean_error": err.mean().item(),
        "std_error": err.std().item()
    }


def percentage_within_threshold(y_true: torch.Tensor, y_pred: torch.Tensor, threshold: float = 0.5) -> float:
    """Calculate percentage of predictions within threshold of true values."""
    y_true = y_true.detach().float().view(-1)
    y_pred = y_pred.detach().float().view(-1)
    
    within_threshold = (torch.abs(y_pred - y_true) <= threshold).float().mean()
    return within_threshold.item()

print("✓ Regression metrics defined")

## Classification Metrics

In [ ]:
def _to_labels(y_pred: torch.Tensor) -> torch.Tensor:
    """
    Convert logits/probs to labels if needed.
    If y_pred is (N, C) -> argmax to (N,)
    If y_pred is (N,) -> assume already labels
    """
    if y_pred.dim() == 2:
        return torch.argmax(y_pred, dim=1)
    return y_pred.view(-1)


def accuracy(y_true: torch.Tensor, y_pred: torch.Tensor) -> float:
    """
    Accuracy for classification.
    
    Args:
        y_true: True labels (N,) long
        y_pred: Logits (N, C) or labels (N,)
        
    Returns:
        Accuracy as float
    """
    y_true = y_true.detach().view(-1).long()
    y_hat = _to_labels(y_pred.detach()).view(-1).long()
    return (y_hat == y_true).float().mean().item()


def macro_f1(y_true: torch.Tensor, y_pred: torch.Tensor, num_classes: int = None, eps: float = 1e-9) -> float:
    """
    Macro F1 score without sklearn dependency.
    
    Args:
        y_true: True labels (N,) long
        y_pred: Logits (N, C) or labels (N,)
        num_classes: Number of classes (auto-detected if None)
        eps: Small epsilon for numerical stability
        
    Returns:
        Macro F1 score as float
    """
    y_true = y_true.detach().view(-1).long()
    y_hat = _to_labels(y_pred.detach()).view(-1).long()

    if num_classes is None:
        num_classes = int(torch.max(torch.cat([y_true, y_hat])).item()) + 1

    f1_sum = 0.0
    for c in range(num_classes):
        tp = ((y_hat == c) & (y_true == c)).sum().float()
        fp = ((y_hat == c) & (y_true != c)).sum().float()
        fn = ((y_hat != c) & (y_true == c)).sum().float()

        precision = tp / (tp + fp + eps)
        recall = tp / (tp + fn + eps)
        f1 = 2.0 * precision * recall / (precision + recall + eps)
        f1_sum += f1.item()

    return f1_sum / float(num_classes)


def per_class_metrics(y_true: torch.Tensor, y_pred: torch.Tensor, class_names: List[str] = None) -> Dict:
    """Calculate per-class precision, recall, and F1 scores."""
    y_true = y_true.detach().view(-1).long()
    y_hat = _to_labels(y_pred.detach()).view(-1).long()
    
    num_classes = int(torch.max(torch.cat([y_true, y_hat])).item()) + 1
    
    if class_names is None:
        class_names = [f"Class_{i}" for i in range(num_classes)]
    
    metrics = {}
    
    for c in range(num_classes):
        tp = ((y_hat == c) & (y_true == c)).sum().float()
        fp = ((y_hat == c) & (y_true != c)).sum().float()
        fn = ((y_hat != c) & (y_true == c)).sum().float()
        tn = ((y_hat != c) & (y_true != c)).sum().float()
        
        precision = tp / (tp + fp + 1e-9)
        recall = tp / (tp + fn + 1e-9)
        f1 = 2.0 * precision * recall / (precision + recall + 1e-9)
        
        class_name = class_names[c] if c < len(class_names) else f"Class_{c}"
        
        metrics[class_name] = {
            "precision": precision.item(),
            "recall": recall.item(),
            "f1": f1.item(),
            "support": int((y_true == c).sum().item())
        }
    
    return metrics

print("✓ Classification metrics defined")

## Visualization Functions

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names, title="Confusion Matrix", figsize=(8, 6)):
    """Plot confusion matrix for classification results."""
    y_true_np = y_true.detach().cpu().numpy() if torch.is_tensor(y_true) else y_true
    y_pred_np = _to_labels(y_pred).detach().cpu().numpy() if torch.is_tensor(y_pred) else y_pred
    
    cm = confusion_matrix(y_true_np, y_pred_np)
    
    plt.figure(figsize=figsize)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()


def plot_regression_results(y_true, y_pred, title="Height Prediction Results", figsize=(12, 4)):
    """Plot regression results with scatter plot and metrics."""
    y_true_np = y_true.detach().cpu().numpy() if torch.is_tensor(y_true) else y_true
    y_pred_np = y_pred.detach().cpu().numpy() if torch.is_tensor(y_pred) else y_pred
    
    metrics = regression_metrics(torch.tensor(y_true_np), torch.tensor(y_pred_np))
    
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    # Scatter plot
    axes[0].scatter(y_true_np, y_pred_np, alpha=0.6, s=50)
    min_val = min(y_true_np.min(), y_pred_np.min())
    max_val = max(y_true_np.max(), y_pred_np.max())
    axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8)
    axes[0].set_xlabel('True Height (m)')
    axes[0].set_ylabel('Predicted Height (m)')
    axes[0].set_title(f'Predictions vs Truth\nR² = {metrics["r2"]:.3f}')
    axes[0].grid(True, alpha=0.3)
    
    # Error distribution
    errors = y_pred_np - y_true_np
    axes[1].hist(errors, bins=20, alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('Prediction Error (m)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'Error Distribution\nMAE = {metrics["mae"]:.3f}m')
    axes[1].axvline(0, color='red', linestyle='--', alpha=0.8)
    axes[1].grid(True, alpha=0.3)
    
    # Error vs true values
    axes[2].scatter(y_true_np, errors, alpha=0.6, s=50)
    axes[2].set_xlabel('True Height (m)')
    axes[2].set_ylabel('Prediction Error (m)')
    axes[2].set_title(f'Error vs True Values\nRMSE = {metrics["rmse"]:.3f}m')
    axes[2].axhline(0, color='red', linestyle='--', alpha=0.8)
    axes[2].grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()
    
    return metrics


def plot_training_metrics(train_losses, val_losses, train_metrics, val_metrics, figsize=(15, 10)):
    """Plot comprehensive training curves."""
    epochs = range(1, len(train_losses) + 1)
    
    fig, axes = plt.subplots(2, 3, figsize=figsize)
    
    # Loss curves
    axes[0, 0].plot(epochs, train_losses, 'b-', label='Train', linewidth=2)
    axes[0, 0].plot(epochs, val_losses, 'r-', label='Validation', linewidth=2)
    axes[0, 0].set_title('Loss Curves')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # MAE curves
    train_mae = [m['mae'] for m in train_metrics]
    val_mae = [m['mae'] for m in val_metrics]
    axes[0, 1].plot(epochs, train_mae, 'b-', label='Train', linewidth=2)
    axes[0, 1].plot(epochs, val_mae, 'r-', label='Validation', linewidth=2)
    axes[0, 1].set_title('Height MAE')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('MAE (meters)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Wave type accuracy
    train_wt_acc = [m.get('wave_type_acc', 0) for m in train_metrics]
    val_wt_acc = [m.get('wave_type_acc', 0) for m in val_metrics]
    axes[0, 2].plot(epochs, train_wt_acc, 'b-', label='Train', linewidth=2)
    axes[0, 2].plot(epochs, val_wt_acc, 'r-', label='Validation', linewidth=2)
    axes[0, 2].set_title('Wave Type Accuracy')
    axes[0, 2].set_xlabel('Epoch')
    axes[0, 2].set_ylabel('Accuracy')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # Direction accuracy
    train_dir_acc = [m.get('direction_acc', 0) for m in train_metrics]
    val_dir_acc = [m.get('direction_acc', 0) for m in val_metrics]
    axes[1, 0].plot(epochs, train_dir_acc, 'b-', label='Train', linewidth=2)
    axes[1, 0].plot(epochs, val_dir_acc, 'r-', label='Validation', linewidth=2)
    axes[1, 0].set_title('Direction Accuracy')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # R² score
    train_r2 = [m.get('r2', 0) for m in train_metrics]
    val_r2 = [m.get('r2', 0) for m in val_metrics]
    axes[1, 1].plot(epochs, train_r2, 'b-', label='Train', linewidth=2)
    axes[1, 1].plot(epochs, val_r2, 'r-', label='Validation', linewidth=2)
    axes[1, 1].set_title('R² Score')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('R²')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # Combined F1 scores
    train_wt_f1 = [m.get('wave_type_macro_f1', 0) for m in train_metrics]
    val_wt_f1 = [m.get('wave_type_macro_f1', 0) for m in val_metrics]
    train_dir_f1 = [m.get('direction_macro_f1', 0) for m in train_metrics]
    val_dir_f1 = [m.get('direction_macro_f1', 0) for m in val_metrics]
    
    axes[1, 2].plot(epochs, train_wt_f1, 'b-', label='Train WT F1', linewidth=2)
    axes[1, 2].plot(epochs, val_wt_f1, 'r-', label='Val WT F1', linewidth=2)
    axes[1, 2].plot(epochs, train_dir_f1, 'b--', label='Train Dir F1', linewidth=2)
    axes[1, 2].plot(epochs, val_dir_f1, 'r--', label='Val Dir F1', linewidth=2)
    axes[1, 2].set_title('Macro F1 Scores')
    axes[1, 2].set_xlabel('Epoch')
    axes[1, 2].set_ylabel('F1 Score')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("✓ Visualization functions defined")

## Example Usage and Testing

In [ ]:
# Create dummy data for testing
def create_dummy_predictions(batch_size=32, num_wave_types=5, num_directions=3):
    """Create dummy predictions for testing metrics."""
    
    # Create realistic predictions
    pred_h = torch.randn(batch_size, 1) * 0.5 + 1.5  # Heights around 1.5m
    pred_wt = torch.randn(batch_size, num_wave_types)
    pred_dir = torch.randn(batch_size, num_directions)

    # Create corresponding targets
    y_h = pred_h + torch.randn(batch_size, 1) * 0.3  # Add some noise
    y_wt = torch.randint(0, num_wave_types, (batch_size,))
    y_dir = torch.randint(0, num_directions, (batch_size,))
    
    # Sample weights (confidence)
    sample_w = torch.rand(batch_size, 1) * 0.5 + 0.5  # Between 0.5 and 1.0
    
    return pred_h, pred_wt, pred_dir, y_h, y_wt, y_dir, sample_w

# Generate test data
print("Creating dummy data for testing...")
pred_h, pred_wt, pred_dir, y_h, y_wt, y_dir, sample_w = create_dummy_predictions()

print(f"✓ Created dummy data:")
print(f"  Batch size: {pred_h.shape[0]}")
print(f"  Height predictions shape: {pred_h.shape}")
print(f"  Wave type predictions shape: {pred_wt.shape}")
print(f"  Direction predictions shape: {pred_dir.shape}")

In [ ]:
# Test loss function
print("Testing multi-task loss function...")

loss_fn = MultiTaskLoss(w_height=1.0, w_wave_type=1.0, w_direction=1.0)
total_loss, loss_dict = loss_fn(pred_h, pred_wt, pred_dir, y_h, y_wt, y_dir, sample_w)

print(f"\n📊 Loss Results:")
print(f"Total loss: {total_loss.item():.4f}")
print(f"Height loss: {loss_dict['loss_h']:.4f}")
print(f"Wave type loss: {loss_dict['loss_wt']:.4f}")
print(f"Direction loss: {loss_dict['loss_dir']:.4f}")

# Test with different weights
print("\nTesting with different task weights...")
loss_fn_weighted = MultiTaskLoss(w_height=2.0, w_wave_type=1.0, w_direction=0.5)
total_loss_w, loss_dict_w = loss_fn_weighted(pred_h, pred_wt, pred_dir, y_h, y_wt, y_dir, sample_w)

print(f"Weighted total loss: {total_loss_w.item():.4f}")
print(f"Height contribution: {loss_dict_w['loss_h'] * 2.0:.4f}")
print(f"Wave type contribution: {loss_dict_w['loss_wt'] * 1.0:.4f}")
print(f"Direction contribution: {loss_dict_w['loss_dir'] * 0.5:.4f}")

In [ ]:
# Test regression metrics
print("Testing regression metrics...")

reg_metrics = regression_metrics(y_h, pred_h)
print(f"\n📏 Regression Metrics:")
for metric, value in reg_metrics.items():
    print(f"  {metric.upper()}: {value:.4f}")

# Test threshold accuracy
within_05 = percentage_within_threshold(y_h, pred_h, threshold=0.5)
within_03 = percentage_within_threshold(y_h, pred_h, threshold=0.3)
print(f"\n🎯 Threshold Accuracy:")
print(f"  Within 0.5m: {within_05:.1%}")
print(f"  Within 0.3m: {within_03:.1%}")

In [ ]:
# Test classification metrics
print("Testing classification metrics...")

wt_acc = accuracy(y_wt, pred_wt)
dir_acc = accuracy(y_dir, pred_dir)
wt_f1 = macro_f1(y_wt, pred_wt, pred_wt.shape[1])
dir_f1 = macro_f1(y_dir, pred_dir, pred_dir.shape[1])

print(f"\n🎯 Classification Metrics:")
print(f"Wave type accuracy: {wt_acc:.4f}")
print(f"Direction accuracy: {dir_acc:.4f}")
print(f"Wave type macro F1: {wt_f1:.4f}")
print(f"Direction macro F1: {dir_f1:.4f}")

# Test per-class metrics
wave_types = ["beach_break", "reef_break", "point_break", "closeout", "a_frame"]
directions = ["left", "right", "both"]

wt_per_class = per_class_metrics(y_wt, pred_wt, wave_types)
dir_per_class = per_class_metrics(y_dir, pred_dir, directions)

print(f"\n📊 Per-Class Wave Type Metrics:")
for class_name, metrics in wt_per_class.items():
    print(f"  {class_name}: F1={metrics['f1']:.3f}, Support={metrics['support']}")

print(f"\n📊 Per-Class Direction Metrics:")
for class_name, metrics in dir_per_class.items():
    print(f"  {class_name}: F1={metrics['f1']:.3f}, Support={metrics['support']}")

## Visualization Examples

In [ ]:
# Visualize regression results
print("Visualizing regression results...")
reg_metrics_vis = plot_regression_results(y_h, pred_h, "Example Height Predictions")

print(f"\nVisualization metrics:")
for metric, value in reg_metrics_vis.items():
    print(f"  {metric}: {value:.4f}")

In [ ]:
# Visualize confusion matrices
print("Visualizing confusion matrices...")

# Wave type confusion matrix
plot_confusion_matrix(y_wt, pred_wt, wave_types, "Wave Type Predictions")

# Direction confusion matrix
plot_confusion_matrix(y_dir, pred_dir, directions, "Direction Predictions")

In [ ]:
# Create dummy training history for visualization
print("Creating example training curves...")

# Simulate training history
epochs = 20
train_losses = [2.5 - 0.1*i + 0.05*np.random.randn() for i in range(epochs)]
val_losses = [2.7 - 0.08*i + 0.08*np.random.randn() for i in range(epochs)]

train_metrics = []
val_metrics = []

for i in range(epochs):
    # Simulate improving metrics
    train_metrics.append({
        'mae': 0.8 - 0.02*i + 0.01*np.random.randn(),
        'r2': 0.1 + 0.03*i + 0.01*np.random.randn(),
        'wave_type_acc': 0.3 + 0.02*i + 0.01*np.random.randn(),
        'direction_acc': 0.4 + 0.015*i + 0.01*np.random.randn(),
        'wave_type_macro_f1': 0.25 + 0.02*i + 0.01*np.random.randn(),
        'direction_macro_f1': 0.35 + 0.015*i + 0.01*np.random.randn()
    })
    
    val_metrics.append({
        'mae': 0.85 - 0.015*i + 0.015*np.random.randn(),
        'r2': 0.05 + 0.025*i + 0.015*np.random.randn(),
        'wave_type_acc': 0.25 + 0.018*i + 0.015*np.random.randn(),
        'direction_acc': 0.35 + 0.012*i + 0.015*np.random.randn(),
        'wave_type_macro_f1': 0.2 + 0.018*i + 0.015*np.random.randn(),
        'direction_macro_f1': 0.3 + 0.012*i + 0.015*np.random.randn()
    })

# Plot training curves
plot_training_metrics(train_losses, val_losses, train_metrics, val_metrics)

print("✓ Training curves visualization complete")

## Comprehensive Evaluation Function

In [ ]:
def evaluate_model_comprehensive(pred_h, pred_wt, pred_dir, y_h, y_wt, y_dir, 
                               wave_type_names=None, direction_names=None, 
                               visualize=True):
    """Comprehensive model evaluation with all metrics and visualizations."""
    
    print("Comprehensive Model Evaluation")
    print("=" * 50)
    
    # Default class names
    if wave_type_names is None:
        wave_type_names = ["beach_break", "reef_break", "point_break", "closeout", "a_frame"]
    if direction_names is None:
        direction_names = ["left", "right", "both"]
    
    # Regression metrics
    reg_metrics = regression_metrics(y_h, pred_h)
    within_05 = percentage_within_threshold(y_h, pred_h, 0.5)
    within_03 = percentage_within_threshold(y_h, pred_h, 0.3)
    
    print(f"\n📏 Height Regression Results:")
    print(f"  MAE: {reg_metrics['mae']:.4f}m")
    print(f"  RMSE: {reg_metrics['rmse']:.4f}m")
    print(f"  R²: {reg_metrics['r2']:.4f}")
    print(f"  Within 0.5m: {within_05:.1%}")
    print(f"  Within 0.3m: {within_03:.1%}")
    
    # Classification metrics
    wt_acc = accuracy(y_wt, pred_wt)
    dir_acc = accuracy(y_dir, pred_dir)
    wt_f1 = macro_f1(y_wt, pred_wt, pred_wt.shape[1])
    dir_f1 = macro_f1(y_dir, pred_dir, pred_dir.shape[1])
    
    print(f"\n🌊 Wave Type Classification:")
    print(f"  Accuracy: {wt_acc:.4f}")
    print(f"  Macro F1: {wt_f1:.4f}")
    
    print(f"\n🧭 Direction Classification:")
    print(f"  Accuracy: {dir_acc:.4f}")
    print(f"  Macro F1: {dir_f1:.4f}")
    
    # Per-class metrics
    wt_per_class = per_class_metrics(y_wt, pred_wt, wave_type_names)
    dir_per_class = per_class_metrics(y_dir, pred_dir, direction_names)
    
    print(f"\n📊 Detailed Wave Type Performance:")
    for class_name, metrics in wt_per_class.items():
        print(f"  {class_name:12}: P={metrics['precision']:.3f} R={metrics['recall']:.3f} F1={metrics['f1']:.3f} (n={metrics['support']})")
    
    print(f"\n📊 Detailed Direction Performance:")
    for class_name, metrics in dir_per_class.items():
        print(f"  {class_name:12}: P={metrics['precision']:.3f} R={metrics['recall']:.3f} F1={metrics['f1']:.3f} (n={metrics['support']})")
    
    # Overall score
    overall_score = (reg_metrics['r2'] + wt_f1 + dir_f1) / 3
    print(f"\n🎯 Overall Performance Score: {overall_score:.4f}")
    print(f"   (Average of R², Wave Type F1, Direction F1)")
    
    # Visualizations
    if visualize:
        print(f"\n📈 Generating visualizations...")
        plot_regression_results(y_h, pred_h, "Model Height Predictions")
        plot_confusion_matrix(y_wt, pred_wt, wave_type_names, "Wave Type Confusion Matrix")
        plot_confusion_matrix(y_dir, pred_dir, direction_names, "Direction Confusion Matrix")
    
    # Return summary
    return {
        'regression': reg_metrics,
        'classification': {
            'wave_type_acc': wt_acc,
            'direction_acc': dir_acc,
            'wave_type_f1': wt_f1,
            'direction_f1': dir_f1
        },
        'per_class': {
            'wave_type': wt_per_class,
            'direction': dir_per_class
        },
        'overall_score': overall_score,
        'threshold_accuracy': {
            'within_0.5m': within_05,
            'within_0.3m': within_03
        }
    }

# Run comprehensive evaluation
print("Running comprehensive evaluation on dummy data...")
evaluation_results = evaluate_model_comprehensive(
    pred_h, pred_wt, pred_dir, y_h, y_wt, y_dir,
    wave_type_names=["beach_break", "reef_break", "point_break", "closeout", "a_frame"],
    direction_names=["left", "right", "both"],
    visualize=True
)

print("\n✅ Comprehensive evaluation complete!")

## Summary

This notebook provides a complete, standalone loss and metrics system for SwellSight:

### ✅ **What we accomplished:**
1. **Multi-Task Loss Function**: Combines regression and classification losses with task weighting
2. **Comprehensive Metrics**: Regression (MAE, RMSE, R²) and classification (accuracy, F1) metrics
3. **Advanced Visualizations**: Training curves, confusion matrices, regression plots
4. **Per-Class Analysis**: Detailed precision, recall, F1 for each class
5. **Evaluation Framework**: Complete model assessment with visualizations
6. **Threshold Analysis**: Percentage of predictions within error thresholds

### 🔧 **Key Features:**
- **Sample Weighting**: Incorporates confidence scores into loss computation
- **Task Balancing**: Adjustable weights for different tasks (height, type, direction)
- **Robust Metrics**: Handles edge cases and provides numerical stability
- **Rich Visualizations**: Publication-ready plots and analysis
- **Comprehensive Evaluation**: Single function for complete model assessment

### 📊 **Metrics Included:**
- **Regression**: MAE, MSE, RMSE, R², mean/std error, threshold accuracy
- **Classification**: Accuracy, macro F1, per-class precision/recall/F1
- **Loss Components**: Individual task losses with weighting
- **Training Monitoring**: Multi-metric training curve visualization

### 🚀 **Usage:**
- Use `MultiTaskLoss` for training with sample weighting
- Use individual metric functions for evaluation
- Use `evaluate_model_comprehensive` for complete assessment
- Use visualization functions for analysis and reporting

**This notebook is completely standalone and includes all necessary functionality for loss computation and model evaluation!**